In [1]:
# 数值特征
# 离散值处理

import pandas as pd
import numpy as np

In [11]:
vg_df = pd.read_csv('D:/user_profile/python/machine-learning/dataset/vgsales.csv',encoding='iso8859-1')
vg_df[['Name', 'Platform', 'Year', 'Genre', 'Publisher']].iloc[1:7]

,Name,Platform,Year,Genre,Publisher
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo
5,Tetris,GB,1989.0,Puzzle,Nintendo
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo


In [12]:
genre = np.unique(vg_df['Genre']) # 获取游戏类型的唯一值
genre

array(['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy'], dtype=object)

In [13]:
# LabelEncoder 对不连续的数字或者文本进行编号

from sklearn.preprocessing import LabelEncoder

gle = LabelEncoder()
genre_labels = gle.fit_transform(vg_df['Genre'])
genre_mapping = {index: label for index,label in enumerate(gle.classes_)}
genre_mapping

{0: 'Action',
 1: 'Adventure',
 2: 'Fighting',
 3: 'Misc',
 4: 'Platform',
 5: 'Puzzle',
 6: 'Racing',
 7: 'Role-Playing',
 8: 'Shooter',
 9: 'Simulation',
 10: 'Sports',
 11: 'Strategy'}

In [14]:
vg_df['GenreLabel'] = genre_labels
vg_df[['Name', 'Platform', 'Year', 'Genre', 'Publisher','GenreLabel']].iloc[1:7]

,Name,Platform,Year,Genre,Publisher,GenreLabel
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,4
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,6
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,10
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,7
5,Tetris,GB,1989.0,Puzzle,Nintendo,5
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,4


In [21]:
# One-hot Encoding 用于将表示分类的数据扩维

poke_df = pd.read_csv('D:/user_profile/python/machine-learning/dataset/Pokemon.csv',encoding='utf8')
poke_df = poke_df.sample(random_state=1,frac=1).reset_index(drop=True)
np.unique(poke_df['Generation'])

gen_ord_map = {'Gen 1': 1, 'Gen 2': 2, 'Gen 3': 3, 
               'Gen 4': 4, 'Gen 5': 5, 'Gen 6': 6}
poke_df['GenerationLabel'] = poke_df['Generation'].map(gen_ord_map)
poke_df[['Name','Generation','GenerationLabel']].iloc[4:10]

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
gen_le = LabelEncoder()
gen_labels = gen_le.fit_transform(poke_df['Generation'])
poke_df['Gen_Label'] = gen_labels

leg_le = LabelEncoder()
leg_labels = leg_le.fit_transform(poke_df['Legendary'])
poke_df['Leg_Label'] = leg_labels

poke_df_sub = poke_df[['Name','Generation','Gen_Label','Legendary','Leg_Label']]
poke_df_sub.iloc[4:10]

,Name,Generation,Gen_Label,Legendary,Leg_Label
4,Octillery,Gen 2,1,False,0
5,Helioptile,Gen 6,5,False,0
6,Dialga,Gen 4,3,True,1
7,DeoxysDefense Forme,Gen 3,2,True,1
8,Rapidash,Gen 1,0,False,0
9,Swanna,Gen 5,4,False,0


In [28]:
gen_ohe = OneHotEncoder()
gen_feature_arr = gen_ohe.fit_transform(poke_df[['Gen_Label']]).toarray()     # 将数组扩充维度
print(gen_feature_arr)
gen_feature_label = list(gen_le.classes_)
print(gen_feature_label)
gen_features = pd.DataFrame(gen_feature_arr,columns=gen_feature_label)

leg_ohe = OneHotEncoder()
leg_feature_arr = leg_ohe.fit_transform(poke_df[['Leg_Label']]).toarray()
print(leg_feature_arr)
leg_feature_label = ['Legendary_'+str(cls_label) for cls_label in leg_le.classes_]
print(leg_feature_label)
leg_features = pd.DataFrame(leg_feature_arr,columns=leg_feature_label)


[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
['Gen 1', 'Gen 2', 'Gen 3', 'Gen 4', 'Gen 5', 'Gen 6']
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
['Legendary_False', 'Legendary_True']


In [29]:
poke_df_ohe = pd.concat([poke_df_sub,gen_features,leg_features],axis=1)
columns = sum([['Name','Generation','Gen_Label'],gen_feature_label,
               ['Legendary','Leg_Label'],leg_feature_label],[])
poke_df_ohe[columns].iloc[4:10]

,Name,Generation,Gen_Label,Gen 1,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6,Legendary,Leg_Label,Legendary_False,Legendary_True
4,Octillery,Gen 2,1,0.0,1.0,0.0,0.0,0.0,0.0,False,0,1.0,0.0
5,Helioptile,Gen 6,5,0.0,0.0,0.0,0.0,0.0,1.0,False,0,1.0,0.0
6,Dialga,Gen 4,3,0.0,0.0,0.0,1.0,0.0,0.0,True,1,0.0,1.0
7,DeoxysDefense Forme,Gen 3,2,0.0,0.0,1.0,0.0,0.0,0.0,True,1,0.0,1.0
8,Rapidash,Gen 1,0,1.0,0.0,0.0,0.0,0.0,0.0,False,0,1.0,0.0
9,Swanna,Gen 5,4,0.0,0.0,0.0,0.0,1.0,0.0,False,0,1.0,0.0


In [30]:
# dummy转换成虚拟指标

gen_dummy_features = pd.get_dummies(poke_df['Generation'],drop_first=True)     # 去除第一行
pd.concat([poke_df[['Name','Generation']],gen_dummy_features],axis=1).iloc[4:10]

gen_onehot_features = pd.get

,Name,Generation,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6
4,Octillery,Gen 2,1,0,0,0,0
5,Helioptile,Gen 6,0,0,0,0,1
6,Dialga,Gen 4,0,0,1,0,0
7,DeoxysDefense Forme,Gen 3,0,1,0,0,0
8,Rapidash,Gen 1,0,0,0,0,0
9,Swanna,Gen 5,0,0,0,1,0
